## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-09-04-14-51-56 +0000,nypost,DC Jewish Museum terror suspect Elias Rodrigue...,https://nypost.com/2025/09/04/us-news/dc-jewis...
1,2025-09-04-14-51-13 +0000,nyt,2 Dead as Russia Strikes Danish Demining Group...,https://www.nytimes.com/2025/09/04/world/europ...
2,2025-09-04-14-46-20 +0000,nyt,"Edgar Feuchtwanger, Who Wrote About Being Hitl...",https://www.nytimes.com/2025/09/04/world/europ...
3,2025-09-04-14-45-48 +0000,bbc,PM refuses to say if he will sack Rayner if sh...,https://www.bbc.com/news/articles/ce321d2n45vo...
4,2025-09-04-14-44-41 +0000,bbc,Legendary Italian designer Giorgio Armani dies,https://www.bbc.com/news/articles/ceq25w34lv4o...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2314/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
98,trump,59
150,up,18
477,new,15
424,putin,14
453,china,13


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
180,2025-09-04-02-55-30 +0000,nypost,Trump admin demands Supreme Court rule quickly...,https://nypost.com/2025/09/03/us-news/trump-ad...,128
195,2025-09-04-01-00-00 +0000,wsj,President Trump’s campaign to bring the Federa...,https://www.wsj.com/economy/central-banking/tr...,128
279,2025-09-03-19-50-17 +0000,nypost,Trump rips reporter for saying he’s taken ‘no ...,https://nypost.com/2025/09/03/us-news/trump-ri...,126
177,2025-09-04-03-23-00 +0000,wsj,The Trump administration asked the Supreme Cou...,https://www.wsj.com/politics/policy/trump-admi...,124
330,2025-09-03-16-36-24 +0000,nypost,"Trump says Putin, Xi were just putting on a sh...",https://nypost.com/2025/09/03/us-news/trump-sa...,116


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
180,128,2025-09-04-02-55-30 +0000,nypost,Trump admin demands Supreme Court rule quickly...,https://nypost.com/2025/09/03/us-news/trump-ad...
195,69,2025-09-04-01-00-00 +0000,wsj,President Trump’s campaign to bring the Federa...,https://www.wsj.com/economy/central-banking/tr...
279,67,2025-09-03-19-50-17 +0000,nypost,Trump rips reporter for saying he’s taken ‘no ...,https://nypost.com/2025/09/03/us-news/trump-ri...
231,59,2025-09-03-22-33-42 +0000,nypost,JD and Usha Vance pay their respects to victim...,https://nypost.com/2025/09/03/us-news/jd-and-u...
255,55,2025-09-03-21-02-22 +0000,nypost,China’s innovation speeds ahead while America ...,https://nypost.com/2025/09/03/world-news/china...
10,46,2025-09-04-14-40-18 +0000,nyt,Portugal Searches for Cause of Lisbon Funicula...,https://www.nytimes.com/2025/09/04/world/europ...
330,43,2025-09-03-16-36-24 +0000,nypost,"Trump says Putin, Xi were just putting on a sh...",https://nypost.com/2025/09/03/us-news/trump-sa...
94,39,2025-09-04-11-00-00 +0000,wsj,U.S. Citizenship and Immigration Services is f...,https://www.wsj.com/politics/policy/agency-tha...
239,38,2025-09-03-22-07-00 +0000,wsj,Florida is proposing to eliminate all vaccine ...,https://www.wsj.com/health/florida-seeks-to-en...
191,38,2025-09-04-01-19-00 +0000,wsj,Trump Administration’s Cuts to Harvard Funding...,https://www.wsj.com/us-news/education/harvard-...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
